In [ ]:
from pysqlcipher3 import dbapi2 as sql

import json
import uuid
from cryptography.hazmat.primitives import hashes
from cryptography.hazmat.backends import default_backend
import base64

import numpy as np
import pandas as pd

SIGNAL_HOME = "/home/yelkhadiri/.config/Signal"

DATABASE_FILE = SIGNAL_HOME + "/sql/db.sqlite"
CONFIG_FILE = SIGNAL_HOME + "/config.json"

with open(CONFIG_FILE, 'r') as f:
    db_key = json.load(f)["key"]

In [ ]:
conn = sql.connect(DATABASE_FILE)
c = conn.cursor()
c.execute(""" PRAGMA key = "x'{}'"; """.format(db_key))

In [ ]:
c.execute("SELECT name FROM sqlite_master WHERE type='table' ORDER BY name;")
available_tables=(c.fetchall())
available_tables

In [ ]:
df = pd.read_sql("SELECT * FROM {}".format("conversations"), conn)
df

In [ ]:
list(map(lambda x: np.datetime64(x, "ms"), df["active_at"]))
pd.Timestamp("2018-11-11 12:43:12").to_datetime64().astype(np.int64) // 10**6

In [ ]:
print(
    json.dumps(
        json.loads(df[df.json.str.contains('profileAvatar')].iloc[0].json),
        indent=4
    )
)

avatar_path = json.loads(df.iloc[-2].json)["profileAvatar"]["path"]
avatar_hash = json.loads(df.iloc[-2].json)["profileAvatar"]["hash"]

```javascript
async function computeHash(arraybuffer) {
  const hash = await crypto.subtle.digest({ name: 'SHA-512' }, arraybuffer);
  return arrayBufferToBase64(hash);
}

if (avatar && avatar.data) {
avatar = {
  hash: await computeHash(avatar.data),
  path: await writeNewAttachmentData(avatar.data),
};
}
```

avatar hash is sha-512 of avatar data

In [ ]:
with open(SIGNAL_HOME + '/attachments.noindex/' + avatar_path, 'rb') as f:
    digest = hashes.Hash(hashes.SHA512(), backend=default_backend())
    digest.update(f.read())
    comp_hash = base64.b64encode(digest.finalize()).decode('utf-8')
    print("comp_hash == avatar_hash ?", comp_hash == avatar_hash)

In [ ]:
def mk_thread_json(profileId, profileName):
    return json.dumps({
        "active_at": str(np.datetime64('now', 'ms').astype(int)),
        "avatar": None,
        "color": "teal",
        "id": profileId,
        "lastMessage": None,
        "lastMessageStatus": None,
        "name": profileName,
        "profileAvatar": None,
        "profileKey": None,
        "profileName": profileName,
        "timestamp": None,
        "tokens": None,
        "type": "private",
        "unreadCount": 0,
        "verified": 0,
        "version": 2
    })

In [ ]:
profileId = '33111111111'
profileName = 'hello world'
mk_thread_json('33111111111', 'hello world')

In [ ]:
df

In [ ]:
c.execute(
    'INSERT into conversations VALUES (?, ?, ?, ?, ?, ?, ?)',
    (profileId, mk_thread_json(profileId, profileName), np.datetime64('now', 'ms').astype(int), 'private', None, profileName, profileName)
)

In [ ]:
c.execute('DELETE from conversations WHERE id="{}"'.format(profileId))

In [ ]:
conn.commit()

In [ ]:
df = pd.read_sql("SELECT * FROM {}".format("messages"), conn)

In [ ]:
df.columns

In [ ]:
df.iloc[-1]

# Type ??

In [ ]:
print(
    json.dumps(
        json.loads(df.iloc[-1].json),
        indent=4
    )
)

In [ ]:
print(
    json.dumps(
        json.loads(df.iloc[-2].json),
        indent=4
    )
)

In [ ]:
def mk_simple_msg_json(conversationId, source, direction, timestamp, msg):
    return json.dumps({
        "attachments": [],
        "body": msg,
        "contact": [],
        "conversationId": conversationId,
        "decrypted_at": timestamp,
        "errors": [],
        "flags": 0,
        "hasAttachments": 0,
        "id": "e18ba6c3-0e89-4f03-afca-617232eb0f36",
        "quote": null,
        "received_at": timestamp,
        "schemaVersion": 9,
        "sent_at": timestamp,
        "source": source,
        "sourceDevice": 1,
        "timestamp": timestamp,
        "type": direction
    })

In [ ]:
c.execute(
    'INSERT into messages VALUES (?, ?, ?, ?, ?, ?, ?)',
    (profileId, mk_thread_json(profileId, profileName), np.datetime64('now', 'ms').astype(int), 'private', None, profileName, profileName)
)

In [ ]:
print(
    json.dumps(
        json.loads(df[df.json.str.contains('contentType')].iloc[0].json),
        indent=4
    )
)

```javascript
function generateUUID() {
  return uuidv4();
}
```

Ids are basic `uuid.uuid4()`

```javascript
exports.createName = () => {
  const buffer = crypto.randomBytes(32);
  return buffer.toString('hex');
};
```

and attachment filenames are 32 random bytes `np.random.bytes(32).hex()`